In [68]:
from tqdm import tqdm
import json
import pandas as pd
from neo4j import GraphDatabase
import time

In [69]:
class Neo4jConnection:
    
    def __init__(self, uri, user, passwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = passwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [70]:
# Create a connection to Neo4j
conn = Neo4jConnection(uri="bolt://127.0.0.1:7687", 
                       user="neo4j",              
                       passwd="NeoPassword2023")

#### Let's try some things. <br>
You must have a "test" database

In [71]:
querystring = """CREATE (p:Person {name: 'Alice', age: 38, eyes: 'brown'})"""
conn.query(querystring, db = "test")

[]

In [72]:
querystring = """MATCH (p) return p.name, p.age limit 10"""
results = conn.query(querystring, db = "test")

# for n in results:
#     print(n)

df = pd.DataFrame([dict(_) for _ in results])
df.head(20)

,p.name,p.age
0,Alice,38


#### Now we can read in some csv files and create a graph

In [73]:
peopledf = pd.read_csv("people.csv")
friendshipsdf = pd.read_csv("friendships.csv")

In [74]:
for index, row in peopledf.iterrows():
    querystring = "CREATE (p:Person {id:" + str(row['Id']) + ", name:'" + row['Name'] + "'})"
    print(querystring)
    conn.query(querystring, db = "test")

CREATE (p:Person {id:1, name:'John'})
CREATE (p:Person {id:10, name:'Jane'})
CREATE (p:Person {id:234, name:'Fred'})
CREATE (p:Person {id:4893, name:'Mark'})
CREATE (p:Person {id:234943, name:'Anne'})


In [75]:
# See if the nodes were created
querystring = """MATCH (p:Person) return p.name, p.id limit 10"""
result = conn.query(querystring, db = "test")
df = pd.DataFrame([dict(_) for _ in result])
df.head(20)

,p.name,p.id
0,John,1.0
1,Jane,10.0
2,Fred,234.0
3,Mark,4893.0
4,Anne,234943.0
5,Alice,NaN


In [76]:
# Now create the friendships
for index, row in friendshipsdf.iterrows():
    querystring = "MATCH (p1:Person {id:" + str(row['Id1']) + "}), (p2:Person {id:" + str(row['Id2']) + "}) CREATE (p1)-[:FRIENDS_WITH]->(p2)"
    print(querystring)
    conn.query(querystring, db = "test")

MATCH (p1:Person {id:1}), (p2:Person {id:234}) CREATE (p1)-[:FRIENDS_WITH]->(p2)
MATCH (p1:Person {id:10}), (p2:Person {id:4893}) CREATE (p1)-[:FRIENDS_WITH]->(p2)
MATCH (p1:Person {id:234}), (p2:Person {id:1}) CREATE (p1)-[:FRIENDS_WITH]->(p2)
MATCH (p1:Person {id:4893}), (p2:Person {id:234943}) CREATE (p1)-[:FRIENDS_WITH]->(p2)
MATCH (p1:Person {id:234943}), (p2:Person {id:234}) CREATE (p1)-[:FRIENDS_WITH]->(p2)
MATCH (p1:Person {id:234943}), (p2:Person {id:1}) CREATE (p1)-[:FRIENDS_WITH]->(p2)


In [77]:
# See if the relationships were created
querystring = """MATCH (p1:Person)-[r:FRIENDS_WITH]->(p2:Person) return p1.name, p2.name limit 10"""
result = conn.query(querystring, db = "test")
df = pd.DataFrame([dict(_) for _ in result])
df.head(20)

,p1.name,p2.name
0,John,Fred
1,Jane,Mark
2,Fred,John
3,Mark,Anne
4,Anne,Fred
5,Anne,John


In [78]:
# Try another query
querystring = """MATCH (:Person {name:"Anne"})-[:FRIENDS_WITH*2..2]-(p2) RETURN p2.name, count(*) as freq ORDER BY freq DESC"""
result = conn.query(querystring, db = "test")
df = pd.DataFrame([dict(_) for _ in result])
df.head(20)

,p2.name,freq
0,Fred,2
1,John,2
2,Jane,1


In [79]:
conn.close()